In [3]:
import re
import pandas as pd
import requests
import FinanceDataReader as fdr
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [4]:
#특정 날짜 23년 2월 2일 크롤링
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23&wr_id=2190#section1'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml')    

In [167]:
#특정 날짜 23년 2월 2일 크롤링
df = pd.DataFrame(columns=['comment'])
for i in range(len(bs.select('.cmt_contents'))):
    comment = bs.select('.cmt_contents')[i].text
    df.loc[i] = [comment]

In [4]:
#게시판에서 각 게시물 주소얻기 
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml') 

In [200]:
#1페이지 링크 얻기
for i in range(len(bs.select('table>tr'))):
    date = bs.select('table>tr')[i].select('dd')[1].text
    url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [5]:
#해당 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
for page_num in range(1,4):
    url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&page={page_num}'
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
        )
    bs = BeautifulSoup(response.text, 'lxml') 

    for i in range(len(bs.select('table>tr'))):
        date = bs.select('table>tr')[i].select('dd')[1].text
        url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [8]:
#함수화
def get_info_from_url(url):
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
    )
    bs = BeautifulSoup(response.text, 'lxml') 
    return bs

In [9]:
#모든 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
df = pd.DataFrame()
for year in tqdm(range(19,24)):
    for page_num in tqdm(range(1,4)):
        url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list{year}&page={page_num}'
        bs = get_info_from_url(url)  
        info_from_bs = bs.select('table>tr')
        for bs in info_from_bs:
            try :
                date = bs.select('dd')[1].text
                url_link = bs.select('a')[0].get('href')
                bs = get_info_from_url(url_link)
                data_from_url = bs.select('.cmt_contents')

                for comments in data_from_url:
                    comment = comments.text

                    temp = pd.DataFrame({
                        'date': date,
                        'url': url_link,
                        'comment': comment
                        }, index=[0])

                    df = pd.concat([df, temp], axis=0)
            except IndexError:
                print(1)

100%|██████████| 5/5 [02:36<00:00, 31.31s/it]

1
1


In [41]:
#파일로 저장
df.to_csv('data.csv', index=False, encoding='utf-8-sig')

In [5]:
#파일 불러오기
df = pd.read_csv('./data.csv')

In [73]:
#필터링 연습
#매수 포함 가져오기
df = df[df['comment'].str.contains('매수', na =False)]
#현재시간 포함 가져오기
df = df[df['comment'].str.contains('현재시간', na =False)]
df = df[df['comment'].str.contains('▶', na =False)]
#공백 지우기
df['comment']=df.comment.apply(lambda x : re.sub('\\n',' ',x) )


In [6]:
#금일 추천주 필터링
df = df[df['comment'].str.contains('매수', na =False)]
df = df[df['comment'].str.contains('현재시간', na =False)]
df = df[df['comment'].str.contains('▶', na =False)]
df['comment'] = df.comment.apply(lambda x : re.sub('\\n','',x))

In [7]:
#date 를 datetime 형식으로 변환 1
df['date'] = df.date.apply(lambda x : x.replace(".",""))
df['date']= df['date'].astype('str')
df['date'] = df.date.apply(lambda x : datetime.strptime(x,'%Y%m%d'))
df = df.reset_index(drop=True)

In [86]:
#date 를 datetime 형식으로 변환 2
df2 = df 
df2['date'] = pd.to_datetime(df2['date'])

In [81]:
# 예외찾기
df2 = df['slicing_comment'].apply(lambda x : x == [] )
df2[df2]

KeyError: 'slicing_comment'

In [8]:
#예외처리
df.comment[54] = df.comment[54].replace('\xa0',' ')

C:\Users\khlee\AppData\Local\Temp\ipykernel_16176\3069763651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.comment[54] = df.comment[54].replace('\xa0',' ')


In [9]:
#종목 슬라이스
df['slicing_comment'] = df.comment.apply(lambda x : re.findall('\d+분 ([가-힣A-z]+)',x)[0])

In [10]:
#종목명 추출
recommended_stock = df.filter(items=['date','slicing_comment'])
recommended_stock = recommended_stock.rename(columns={'slicing_comment':'stock_name'})

In [31]:
#데이터프레임 한번에 보기
pd.set_option('display.max_rows', None)
#다시 돌리기
pd.options.display.max_rows = 100

In [11]:
recommended_stock

,date,stock_name
0,2019-12-27,에스맥
1,2019-12-27,화천기계
2,2019-12-27,루미마이크로
3,2019-12-26,한국화장품
4,2019-12-26,디앤씨미디어
...,...,...
967,2022-12-22,팬오션
968,2022-12-21,세종메디칼
969,2022-12-21,청담글로벌
970,2022-12-19,노터스


In [12]:
Kospi_df = pd.read_csv('./kospi_code.csv')
Kosdaq_df = pd.read_csv('./kosdaq_code.csv')
kospi = Kospi_df.filter(items= ['단축코드','한글명'])
kosdaq = Kosdaq_df.filter(items= ['단축코드','한글종목명'])

In [15]:
kospi = kospi.rename(columns={'단축코드':'stock_code','한글명' : 'stock_name'})
kosdaq = kosdaq.rename(columns={'단축코드':'stock_code','한글종목명' : 'stock_name'})

In [34]:
kospi_kosdaq.query('stock_code == "041140"')

,stock_code,stock_name


In [19]:
kospi_kosdaq = pd.concat([kospi,kosdaq])

In [20]:
stock = pd.merge(recommended_stock, kospi_kosdaq, how = 'left', on  = 'stock_name')

In [21]:
stock

,date,stock_name,stock_code
0,2019-12-27,에스맥,097780
1,2019-12-27,화천기계,010660
2,2019-12-27,루미마이크로,NaN
3,2019-12-26,한국화장품,123690
4,2019-12-26,디앤씨미디어,263720
...,...,...,...
967,2022-12-22,팬오션,028670
968,2022-12-21,세종메디칼,258830
969,2022-12-21,청담글로벌,362320
970,2022-12-19,노터스,278650


In [24]:
pd.set_option('display.max_rows', None)

In [25]:
# 예외찾기
stock[stock['stock_code'].isnull()]


,date,stock_name,stock_code
2,2019-12-27,루미마이크로,NaN
6,2019-12-26,제낙스,NaN
9,2019-12-24,에스모머티리얼즈,NaN
11,2019-12-23,우성사료,NaN
15,2019-12-20,에스모,NaN
20,2019-12-19,화신테크,NaN
30,2019-12-16,갤럭시아컴즈,NaN
34,2019-12-13,코리아에스이,NaN
44,2019-12-11,화신테크,NaN
47,2019-12-10,현대에이치씨엔,NaN
